# LSTM Training

## imports

In [1]:
import numpy as np
import re
from IPython.display import clear_output

from tensorflow.keras.layers import Dense, LSTM, Input, Embedding, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import LambdaCallback

In [20]:
load_saved_model = True
train_model = True

In [21]:
token_type = 'word'

## data

In [22]:
#load in the text and perform some cleanup

seq_length = 20

filename = "./data/texto.txt"

with open(filename, encoding='utf-8-sig') as f:
    text = f.read()

In [23]:
start_story = '| ' * seq_length
    
text = start_story + text
text = text.lower()
text = text.replace('\n\n\n\n\n', start_story)
text = text.replace('\n', ' ')
text = re.sub('  +', '. ', text).strip()
text = text.replace('..', '.')

text = re.sub('([!"#$%&()*+,-./:;<=>?@[\]^_`{|}~])', r' \1 ', text)
text = re.sub('\s{2,}', ' ', text)

In [24]:
len(text)

3065

In [25]:
text

' | | | | | | | | | | | | | | | | | | | | art . 1 . es delito toda acción u omisión voluntaria penada por la ley . las acciones u omisiones penadas por la ley se reputan siempre voluntarias , a no ser que conste lo contrario . el que cometiere delito será responsable de él e incurrirá en la pena que la ley señale , aunque el mal recaiga sobre persona distinta de aquella a quien se proponía ofender . en tal caso no se tomarán en consideración las circunstancias , no conocidas por el delincuente , que agravarían su responsabilidad ; pero sí aquellas que la atenúen . | | | | | | | | | | | | | | | | | | | | art . 2 . las acciones u omisiones que cometidas con dolo o malicia importarían un delito , constituyen cuasidelito si sólo hay culpa en el que las comete . | | | | | | | | | | | | | | | | | | | | art . 3 . los delitos , atendida su gravedad , se dividen en crímenes , simples delitos y faltas y se califican de tales según la pena que les está asignada en la escala general del art . 21 .

In [26]:

if token_type == 'word':
    tokenizer = Tokenizer(char_level = False, filters = '')
else:
    tokenizer = Tokenizer(char_level = True, filters = '', lower = False)
    
    
tokenizer.fit_on_texts([text])

total_words = len(tokenizer.word_index) + 1

token_list = tokenizer.texts_to_sequences([text])[0]


In [27]:
total_words

205

In [28]:
print(len(token_list))
print(tokenizer.word_index)
print(token_list)

688
{'|': 1, '.': 2, 'la': 3, ',': 4, 'o': 5, 'que': 6, 'de': 7, 'el': 8, 'delito': 9, 'se': 10, 'en': 11, 'art': 12, 'por': 13, 'los': 14, 'a': 15, 'y': 16, 'crimen': 17, 'simple': 18, 'las': 19, 'un': 20, 'ley': 21, 'no': 22, 'su': 23, 'para': 24, 'delitos': 25, 'del': 26, 'cuando': 27, 'u': 28, 'pena': 29, 'sólo': 30, 'ejecución': 31, 'es': 32, 'hay': 33, 'casos': 34, 'conspiración': 35, 'proposición': 36, 'cometer': 37, 'toda': 38, 'acciones': 39, 'omisiones': 40, 'lo': 41, 'tal': 42, 'circunstancias': 43, 'delincuente': 44, 'pero': 45, 'con': 46, 'crímenes': 47, 'simples': 48, 'faltas': 49, 'califican': 50, 'este': 51, 'código': 52, 'república': 53, 'extranjeros': 54, 'sino': 55, 'son': 56, 'punibles': 57, 'frustrado': 58, 'tentativa': 59, 'verifica': 60, 'culpable': 61, 'más': 62, 'personas': 63, '1': 64, 'acción': 65, 'omisión': 66, 'voluntaria': 67, 'penada': 68, 'penadas': 69, 'reputan': 70, 'siempre': 71, 'voluntarias': 72, 'ser': 73, 'conste': 74, 'contrario': 75, 'cometiere

In [29]:
def generate_sequences(token_list, step):
    
    X = []
    y = []

    for i in range(0, len(token_list) - seq_length, step):
        X.append(token_list[i: i + seq_length])
        y.append(token_list[i + seq_length])
    

    y = to_categorical(y, num_classes = total_words)
    
    num_seq = len(X)
    print('Number of sequences:', num_seq, "\n")
    
    return X, y, num_seq

step = 1
seq_length = 20

X, y, num_seq = generate_sequences(token_list, step)

X = np.array(X)
y = np.array(y)


Number of sequences: 668 



In [30]:
X.shape

(668, 20)

In [31]:
y.shape

(668, 205)

## Define the LSTM model

In [32]:
if load_saved_model:
    model = load_model('./saved_models/epoch300_batch-size128.h5')

else:

    n_units = 256
    embedding_size = 100

    text_in = Input(shape = (None,))
    embedding = Embedding(total_words, embedding_size)
    x = embedding(text_in)
    x = LSTM(n_units)(x)
    text_out = Dense(total_words, activation = 'softmax')(x)

    model = Model(text_in, text_out)

    opti = RMSprop(learning_rate = 0.001)
    model.compile(loss='categorical_crossentropy', optimizer=opti)

In [33]:
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding_2 (Embedding)     (None, None, 100)         20500     
                                                                 
 lstm_2 (LSTM)               (None, 256)               365568    
                                                                 
 dense_2 (Dense)             (None, 205)               52685     
                                                                 
Total params: 438,753
Trainable params: 438,753
Non-trainable params: 0
_________________________________________________________________


In [34]:
def sample_with_temp(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)



def generate_text(seed_text, next_words, model, max_sequence_len, temp):
    output_text = seed_text
    
    seed_text = start_story + seed_text
    
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = token_list[-max_sequence_len:]
        token_list = np.reshape(token_list, (1, max_sequence_len))
        
        probs = model.predict(token_list, verbose=0)[0]
        y_class = sample_with_temp(probs, temperature = temp)
        
        if y_class == 0:
            output_word = ''
        else:
            output_word = tokenizer.index_word[y_class]
            
        if output_word == "|":
            break
            
        if token_type == 'word':
            output_text += output_word + ' '
            seed_text += output_word + ' '
        else:
            output_text += output_word + ' '
            seed_text += output_word + ' '
            
            
    return output_text

In [19]:
def on_epoch_end(epoch, logs):
    seed_text = ""
    gen_words = 500

    print('Temp 0.2')
    print (generate_text(seed_text, gen_words, model, seq_length, temp = 0.2))
    print('Temp 0.33')
    print (generate_text(seed_text, gen_words, model, seq_length, temp = 0.33))
    print('Temp 0.5')
    print (generate_text(seed_text, gen_words, model, seq_length, temp = 0.5))
    print('Temp 1.0')
    print (generate_text(seed_text, gen_words, model, seq_length, temp = 1))

    
    
if train_model:
    epochs = 1000
    batch_size = 64
    num_batches = int(len(X) / batch_size)
    callback = LambdaCallback(on_epoch_end=on_epoch_end)
    model.fit(X, y, epochs=epochs, batch_size=batch_size, callbacks = [callback], shuffle = True)




Epoch 1/1000
21/21 [==============================] - ETA: 0s - loss: 4.4713Temp 0.2

Temp 0.33

Temp 0.5

Temp 1.0

21/21 [==============================] - 3s 73ms/step - loss: 4.4713
Epoch 2/1000
20/21 [===========================>..] - ETA: 0s - loss: 3.9202Temp 0.2

Temp 0.33

Temp 0.5

Temp 1.0

21/21 [==============================] - 1s 52ms/step - loss: 3.9213
Epoch 3/1000
21/21 [==============================] - ETA: 0s - loss: 3.7190Temp 0.2

Temp 0.33

Temp 0.5

Temp 1.0

21/21 [==============================] - 1s 56ms/step - loss: 3.7190
Epoch 4/1000
20/21 [===========================>..] - ETA: 0s - loss: 3.5870Temp 0.2

Temp 0.33

Temp 0.5

Temp 1.0

21/21 [==============================] - 1s 52ms/step - loss: 3.6115
Epoch 5/1000
21/21 [==============================] - ETA: 0s - loss: 3.5548Temp 0.2

Temp 0.33

Temp 0.5

Temp 1.0
. . . sólo un simple por , todo los simple delitos conocidas sólo los con o parte la a crimen conspiración la escala la culpable sino aquell

21/21 [==============================] - ETA: 0s - loss: 1.9636Temp 0.2

Temp 0.33

Temp 0.5

Temp 1.0

21/21 [==============================] - 1s 51ms/step - loss: 1.9636
Epoch 29/1000
21/21 [==============================] - ETA: 0s - loss: 1.8639Temp 0.2

Temp 0.33

Temp 0.5
art . 2 . los , simples toda es es los de . la la por , acciones por omisiones y , culpable ley incurrirá incurrirá iniciarse se ejecución la la su el su ley la de a de de de culpable culpable consideración la de incurrirá a crimen simple simple cuando de ley contra que la casos las de a en de la la más . la la tentativa . ha a código más más personas conspiración para ejecución conciertan ejecución ejecución su la la la la la la la ejecución la ejecución ejecución ejecución la ejecución se para ejecución proposición ejecución ejecución su su ejecución su se siempre la la la a de se la incurrirá incurrirá crimen la la simple cuando cuando delincuente prescripciones la está de la la 
Temp 1.0

21/21 [===========

21/21 [==============================] - ETA: 0s - loss: 1.3848Temp 0.2
art . 8 . la ley , no sólo el crimen o simple delito delito , , sólo sólo los por el la la para o . la y la la . dos dos o . 
Temp 0.33
art . 5 . la ley , , castigan delitos , sido los los la república , inclusos los los extranjeros . los delitos iniciarse tales iniciarse del del iniciarse según territorial territorial está en sometidos sometidos sí aquellas código la que . 21 conspiración proposición conspiración conspiración proposición proposición para cometer o o o o simple delito . la se cuando cuando su la la la la . ejecución delito . delito . la cuando la tentativa todo para cometer , crimen o ejecución o simple delito . . los , hechos directos la de las da ejecución o o o simple simple delito . se cuando , el este la de . la . o . 
Temp 0.5
art . 2 . son delito delitos delitos es perpetrados todos los sido en la la república república por por extranjeros , extranjeros delitos judicial del tales adyacente e

KeyboardInterrupt: 

In [20]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None)]            0         
_________________________________________________________________
embedding (Embedding)        (None, None, 100)         20500     
_________________________________________________________________
lstm (LSTM)                  (None, 256)               365568    
_________________________________________________________________
dense (Dense)                (None, 205)               52685     
Total params: 438,753
Trainable params: 438,753
Non-trainable params: 0
_________________________________________________________________


In [41]:
seed_text = "cuando es penado un delito "
gen_words = 20
temp = 1

print (generate_text(seed_text, gen_words, model, seq_length, temp))
print (generate_text(seed_text, gen_words, model, seq_length, temp))
print (generate_text(seed_text, gen_words, model, seq_length, temp))

cuando es penado un delito . 9 . las faltas sólo se castigan cuando han sido consumadas . las inclusos se cometidos el iniciarse a 
cuando es penado un delito . 3 . los delitos , atendida su gravedad , los cuasidelitos , simples delitos y crimen o . directos 
cuando es penado un delito . 3 . los delitos , atendida , gravedad , en crímenes , simples delitos y o obra obra califican 


## Guardar modelo

In [22]:
model.save('./saved_models/prueba1.h5')

## Generar secuencias para la generación de texto sintético a partir de un artículo

In [107]:
def sequences_for_generation(token_list, step):
    
    X = []

    for i in range(0, len(token_list) - seq_length, step):
        X.append(token_list[i: i + seq_length])
    
    num_seq = len(X)
    print('Number of sequences:', num_seq, "\n")
    
    return X, num_seq

step = 5
seq_length = 5

X, num_seq = sequences_for_generation(token_list, step)

#X = np.array(X)

Number of sequences: 137 



In [52]:
print(token_list)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 12, 2, 64, 2, 32, 9, 38, 65, 28, 66, 67, 68, 13, 3, 21, 2, 19, 39, 28, 40, 69, 13, 3, 21, 10, 70, 71, 72, 4, 15, 22, 73, 6, 74, 41, 75, 2, 8, 6, 76, 9, 77, 78, 7, 79, 80, 81, 11, 3, 29, 6, 3, 21, 82, 4, 83, 8, 84, 85, 86, 87, 88, 7, 89, 15, 90, 10, 91, 92, 2, 11, 42, 93, 22, 10, 94, 11, 95, 19, 43, 4, 22, 96, 13, 8, 44, 4, 6, 97, 23, 98, 99, 45, 100, 101, 6, 3, 102, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 12, 2, 103, 2, 19, 39, 28, 40, 6, 104, 46, 105, 5, 106, 107, 20, 9, 4, 108, 109, 110, 30, 33, 111, 11, 8, 6, 19, 112, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 12, 2, 113, 2, 14, 25, 4, 114, 23, 115, 4, 10, 116, 11, 47, 4, 48, 25, 16, 49, 16, 10, 50, 7, 117, 118, 3, 29, 6, 119, 120, 121, 11, 3, 122, 123, 26, 12, 2, 124, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 12, 2, 125, 2, 3, 126, 7, 14, 25, 32, 127, 15, 14, 128, 4, 6, 10, 50, 16, 129, 11, 14, 34, 130, 6, 13

In [94]:
c = 0
i = 0
while c < 7:
    pos = token_list[i:].index(1)
    c = c + 1
    i = i + pos + 20

In [100]:
end7 = i + token_list[i:].index(1)

In [116]:
seq, seq_n = sequences_for_generation([list(t.keys())[list(t.values()).index(x)] for x in token_list[i:end7]], step)

Number of sequences: 17 



In [117]:
[' '.join(x) for x in seq]

['art . 7 . son',
 'punibles , no sólo el',
 'crimen o simple delito consumado',
 ', sino el frustrado y',
 'la tentativa . hay crimen',
 'o simple delito frustrado cuando',
 'el delincuente pone de su',
 'parte todo lo necesario para',
 'que el crimen o simple',
 'delito se consume y esto',
 'no se verifica por causas',
 'independientes de su voluntad .',
 'hay tentativa cuando el culpable',
 'da principio a la ejecución',
 'del crimen o simple delito',
 'por hechos directos , pero',
 'faltan uno o más para']

In [153]:
seq_length = 5
seq, seq_n = sequences_for_generation([list(t.keys())[list(t.values()).index(x)] for x in art1], 5)

Number of sequences: 14 



In [154]:
seq

[['art', '.', '1', '.', 'es'],
 ['delito', 'toda', 'acción', 'u', 'omisión'],
 ['voluntaria', 'penada', 'por', 'la', 'ley'],
 ['.', 'las', 'acciones', 'u', 'omisiones'],
 ['penadas', 'por', 'la', 'ley', 'se'],
 ['reputan', 'siempre', 'voluntarias', ',', 'a'],
 ['no', 'ser', 'que', 'conste', 'lo'],
 ['contrario', '.', 'el', 'que', 'cometiere'],
 ['delito', 'será', 'responsable', 'de', 'él'],
 ['e', 'incurrirá', 'en', 'la', 'pena'],
 ['que', 'la', 'ley', 'señale', ','],
 ['aunque', 'el', 'mal', 'recaiga', 'sobre'],
 ['persona', 'distinta', 'de', 'aquella', 'a'],
 ['quien', 'se', 'proponía', 'ofender', '.']]

In [155]:
[' '.join(x) for x in seq]

['art . 1 . es',
 'delito toda acción u omisión',
 'voluntaria penada por la ley',
 '. las acciones u omisiones',
 'penadas por la ley se',
 'reputan siempre voluntarias , a',
 'no ser que conste lo',
 'contrario . el que cometiere',
 'delito será responsable de él',
 'e incurrirá en la pena',
 'que la ley señale ,',
 'aunque el mal recaiga sobre',
 'persona distinta de aquella a',
 'quien se proponía ofender .']

## Generación de texto sintético a partir de artículos del código penal vigente de Chile

In [54]:
art1 = ['art . 1 . es',
 'delito toda acción u omisión',
 'voluntaria penada por la ley',
 '. las acciones u omisiones',
 'penadas por la ley se',
 'reputan siempre voluntarias , a',
 'no ser que conste lo',
 'contrario . el que cometiere',
 'delito será responsable de él',
 'e incurrirá en la pena',
 'que la ley señale ,',
 'aunque el mal recaiga sobre',
 'persona distinta de aquella a',
 'quien se proponía ofender .']
art7 = ['art . 7 . son',
 'punibles , no sólo el',
 'crimen o simple delito consumado',
 ', sino el frustrado y',
 'la tentativa . hay crimen',
 'o simple delito frustrado cuando',
 'el delincuente pone de su',
 'parte todo lo necesario para',
 'que el crimen o simple',
 'delito se consume y esto',
 'no se verifica por causas',
 'independientes de su voluntad .',
 'hay tentativa cuando el culpable',
 'da principio a la ejecución',
 'del crimen o simple delito',
 'por hechos directos , pero',
 'faltan uno o más para']

In [44]:
for seed in art1:
    print ('- ' + generate_text(seed, gen_words, model, seq_length, temp))

- art . 1 . esdelito es delito toda acción u omisión voluntaria penada por la ley . las acciones u omisiones penadas por la 
- delito toda acción u omisiónfaltas las faltas sólo se castigan cuando han sido consumadas . las acciones . las delincuente el responsabilidad responsabilidad quedan 
- voluntaria penada por la ley. . la ley penal chilena es obligatoria para todos los habitantes de la república , inclusos lo ser de 
- . las acciones u omisiones. 1 . es delito toda acción u omisión voluntaria penada por la república por acciones u contrario por no 
- penadas por la ley sees . la conspiración y proposición para cometer un crimen o un simple delito , sólo son la la éstos 
- reputan siempre voluntarias , au acciones u omisiones que cometidas con dolo o malicia importarían un delito , constituyen , chile ; pero el 
- no ser que conste loes . es delito toda acción u omisión voluntaria penada por la república por acciones u omisiones que la la 
- contrario . el que cometiere. la ley pe

In [46]:
for seed in art7:
    print ('- ' + generate_text(seed, gen_words, model, seq_length, temp))

- art . 7 . sonpunibles son punibles , no sólo el crimen o simple delito consumado , sino el frustrado y la tentativa . 
- punibles , no sólo el. 8 . la conspiración y proposición para cometer un crimen o un simple delito , son crimen delito , 
- crimen o simple delito consumado5 . 9 . las faltas sólo se castigan cuando han sido consumadas . las acciones , ley procedimiento independientes 
- , sino el frustrado yes . los delitos , atendida su gravedad , se dividen en crímenes , simples delitos y del distinta califican 
- la tentativa . hay crimenla . la conspiración y proposición para cometer un crimen o un simple delito , frustrado y en en los 
- o simple delito frustrado cuando. 9 . las faltas sólo se castigan cuando han sido consumadas . las acciones y , aunque sobre él 
- el delincuente pone de su9 . 7 . son punibles , no sólo el simple delito consumado , sino el frustrado cuando frustrado cuando 
- parte todo lo necesario paralos . los crímenes o simples delitos perpetrados fuera 